<a href="https://colab.research.google.com/github/josephxlp/PyTorch100Days/blob/main/W1DAY4_Binary_Image_Classifier_(Cats_vs_Dogs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

W1DAY4: Binary Image Classifier (Cats vs Dogs)

Description:

- Train a binary classifier to distinguish between cat and dog images using CNN
-This project introduces image folders, binary classification, CNN architecture and kaggle


    - Goal: Classify input images as either cat or dog using a CNN.

In [1]:
import os
import shutil
import logging
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import userdata
from tqdm import tqdm

# --- Setup logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Image transformations ---
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# --- Set credentials ---
os.environ['KAGGLE_USERNAME'] = 'josephexeter'
os.environ['KAGGLE_KEY'] = userdata.get('josephexeter')

# --- Directory paths ---
train_dir = '/content/train'
cat_dir = os.path.join(train_dir, 'cat')
dog_dir = os.path.join(train_dir, 'dog')
test_dir = '/content/test'
os.makedirs(test_dir, exist_ok=True)

logging.info("set up main variables")

In [2]:
# --- Download and organize dataset if needed ---
if os.path.exists(cat_dir) and os.path.exists(dog_dir):
    logging.info("Class directories already exist. Skipping setup.")
else:
    logging.info("Downloading and extracting dataset...")
    os.system("kaggle competitions download -c dogs-vs-cats")
    if os.path.exists('dogs-vs-cats.zip'):
        os.system("unzip -q -o dogs-vs-cats.zip")
    if os.path.exists('train.zip'):
        os.system("unzip -q -o train.zip")
    if os.path.exists('test1.zip'):
        os.system("unzip -q -o test1.zip")

    os.makedirs(cat_dir, exist_ok=True)
    os.makedirs(dog_dir, exist_ok=True)

    logging.info("Organizing training images...")
    for filename in os.listdir('/content/train'):
        src = os.path.join('/content/train', filename)
        if filename.startswith('cat.'):
            shutil.move(src, os.path.join(cat_dir, filename))
        elif filename.startswith('dog.'):
            shutil.move(src, os.path.join(dog_dir, filename))

    logging.info("Organizing test images...")
    for filename in os.listdir('/content/test1'):
        shutil.move(os.path.join('/content/test1', filename), os.path.join(test_dir, filename))


logging.info(f"2. Donwloaded dataset:: train and test")

In [3]:
print('# --- Load datasets ---')
dataset = datasets.ImageFolder(train_dir, transform=transform)
class_names = dataset.classes

print('# --- Split into training and validation ---')
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=32, shuffle=False)

# --- Load datasets ---
# --- Split into training and validation ---


In [4]:
print('# --- Custom Dataset for unlabeled test images ---')
class UnlabeledImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_files[idx]

test_ds = UnlabeledImageDataset(test_dir, transform=transform)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

# --- Custom Dataset for unlabeled test images ---


In [5]:
print('# --- Define CNN ---')
class CatDogCNN(nn.Module):
    def __init__(self):
        super(CatDogCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 32 * 32, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 32 * 32)
        return torch.sigmoid(self.fc1(x))

# --- Define CNN ---


In [6]:
print('# --- Training Setup ---')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CatDogCNN().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
logging.info(f"Using device: {device}")
print(f'# --- Training Setup --- {device}')

# --- Training Setup ---
# --- Training Setup --- cuda


In [ ]:
print('# --- Training Loop ---')
for epoch in range(7):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(train_dl, desc=f"Epoch {epoch+1}", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(train_dl)
    logging.info(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")


# --- Training Loop ---


Epoch 5:   7%|▋         | 42/625 [00:03<00:55, 10.50it/s, loss=0.219]

In [ ]:
print('# --- Validation Metrics ---')
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in val_dl:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images).cpu().numpy().flatten()
        preds = (outputs > 0.5).astype(int)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds)

logging.info(f"Validation Accuracy: {accuracy_score(y_true, y_pred):.4f}")
logging.info(f"Precision: {precision_score(y_true, y_pred):.4f}")
logging.info(f"Recall: {recall_score(y_true, y_pred):.4f}")
logging.info(f"F1 Score: {f1_score(y_true, y_pred):.4f}")
logging.info(f"Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")




In [ ]:
print(f"Validation Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"Precision: {precision_score(y_true, y_pred):.4f}")
print(f"Recall: {recall_score(y_true, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")
print(f"Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")

In [ ]:
# --- Predict on Test Set ---
def predict_test(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, filenames in dataloader:
            images = images.to(device)
            outputs = model(images).cpu().numpy().flatten()
            preds = (outputs > 0.5).astype(int)
            for fname, pred in zip(filenames, preds):
                predictions.append((fname, class_names[pred]))
    return pd.DataFrame(predictions, columns=['filename', 'predicted_label'])

# Generate test predictions
test_predictions = predict_test(model, test_dl)
test_predictions.head()

In [ ]:
# build a more complex model
# yolo